In [21]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\nikol\OneDrive\Desktop\project_repos\CMLEval\data\crosslingual\enQ_xA\KazMMLU.parquet.gzip")

exclude = ["Math (High School in kaz)", "Math (High School in rus)"]
df = df.drop(df[df["subject"].isin(exclude)].index)
# df = df.groupby("Language").sample(n=5)
df = df[df["Language"] == "Kazakh"].copy()
print(len(df))
df.head()

538


,dataset_name,subject,question_idx,marked_answer,llm_answer,is_correct,explanation,Question,Option A,Option B,Option C,Option D,Option E,Language
46,KazMMLU,Biology (High School in kaz),0,A,A,True,"The term ""Қуат беретін станциясы"" translates t...",Cell power station,Митохондриялар,Лизосома,Центриоль,Хлоропластар,Ядрошық,Kazakh
47,KazMMLU,Biology (High School in kaz),1,D,D,True,The question asks for the part of the flower w...,Place where all the flower sections are fixed:,Күлте,Тостағанша,Гүл бөлімдері,Гүл табаны,None,Kazakh
48,KazMMLU,Biology (High School in kaz),2,B,B,True,The question asks which structure connects the...,Connects the middle ear cavity and the cavity,Есту сүйекшелері,Есту түтігі,Ұлулы дене,Дабыл жарғағы,None,Kazakh
49,KazMMLU,Biology (High School in kaz),3,A,A,True,The question asks for an important organelle f...,"Important organids of plant and animal cells, ...",Ядро,Цитоплазма,Гольджи апараты,Жасуша,None,Kazakh
50,KazMMLU,Biology (High School in kaz),4,B,B,True,The question asks about the origin of many tub...,"The homeland of many bulk plants, including po...",Орталық америкалық орталық,Андылық (Оңтүстік америкалық) орталық,Шығысазиялық орталық,Оңтүстік-батысазиялық орталық,None,Kazakh


In [40]:
# prompt = """You are an expert at testing multiple-choice questions. Look at the question, and choose the correct answer option from the ones given to you.
# Do not generate your own answer, only pick one from the options given.""" 

prompt = """You are an expert at testing multiple-choice questions. Look at the question, think about it, and choose the correct answer option from the ones given to you.
Write out your thought process, and in the end, put the correct option in square brackets: [A], [B], etc.
One of these answers is correct."""

prompt += """
Question: {question}
Options:
{options}
Answer: """
print(prompt)

You are an expert at testing multiple-choice questions. Look at the question, think about it, and choose the correct answer option from the ones given to you.
Write out your thought process, and in the end, put the correct option in square brackets: [A], [B], etc.
One of these answers is correct.
Question: {question}
Options:
{options}
Answer: 


In [41]:
def create_prompt(row: pd.Series) -> str:
    question = row["Question"]
    options = [row["Option A"], row["Option B"], row["Option C"], row["Option D"], row["Option E"]]
    options = [o for o in options if o]
    options = "\n".join(
        [f"{chr(65 + i)}. {o}".strip() for i, o in enumerate(options)]
    )
    return prompt.format(question=question, options=options)

In [48]:
row = df.iloc[104]
# row = df.loc[928]
print(row.name)
print(row["marked_answer"])
print(create_prompt(row))

583
A
You are an expert at testing multiple-choice questions. Look at the question, think about it, and choose the correct answer option from the ones given to you.
Write out your thought process, and in the end, put the correct option in square brackets: [A], [B], etc.
One of these answers is correct.
Question: The formation of climatic zones on the planet is related to:
Options:
A. Жер бетіне түсетін күн жылуының мөлшеріне
B. Ауа қозғалысына
C. Қысымның таралуына
D. Жауын-шашынның таралуына
Answer: 
